In [1]:
sample = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""

In [2]:
with open('input.txt') as f:
    data = f.read()

In [3]:
import re
import numpy as np
from collections import deque

In [4]:
from tqdm.notebook import tqdm

In [5]:
pattern = re.compile(r"Monkey (\d+)\:\n\s*Starting items: ((?:\d+(?:, )?)+)\n\s*Operation: (new = old . \w+)\n\s*Test: divisible by (\d+)\n\s*If true: throw to monkey (\d+)\n\s*If false: throw to monkey (\d+)")

In [6]:
pattern.findall(data)

[('0', '78, 53, 89, 51, 52, 59, 58, 85', 'new = old * 3', '5', '2', '7'),
 ('1', '64', 'new = old + 7', '2', '3', '6'),
 ('2', '71, 93, 65, 82', 'new = old + 5', '13', '5', '4'),
 ('3', '67, 73, 95, 75, 56, 74', 'new = old + 8', '19', '6', '0'),
 ('4', '85, 91, 90', 'new = old + 4', '11', '3', '1'),
 ('5', '67, 96, 69, 55, 70, 83, 62', 'new = old * 2', '3', '4', '1'),
 ('6', '53, 86, 98, 70, 64', 'new = old + 6', '7', '7', '0'),
 ('7', '88, 64', 'new = old * old', '17', '2', '5')]

In [7]:
op_re = re.compile(r'new = old (.) (\w+)')

In [8]:
def parse_operation(op):
    match = op_re.match(op)
    operator, operand = match.groups()
    if operator == '*':
        if operand == 'old':
            return lambda x: x * x
        else:
            return lambda x: x * int(operand)
    elif operator == '+':
        if operand == 'old':
            return lambda x: x + x
        else:
            return lambda x: x + int(operand)
    else:
        raise AssertError

In [9]:
monkeys = {}

for (monkey, items, operation, test, true, false) in pattern.findall(data):
    monkeys[int(monkey)] = {'items': [], 'operation': parse_operation(operation), 
                            'test': int(test), 'true': int(true), 
                            'false': int(false), 'inspected': 0}
    for item in items.split(', '):
        monkeys[int(monkey)]['items'].append(int(item))

In [10]:
for _ in range(20):
    for monkey in monkeys:
        items = monkeys[monkey]['items']
        while len(items) > 0:
            old = items.pop(0)
            new = monkeys[monkey]['operation'](old)
            new = new // 3
            if new % monkeys[monkey]['test'] == 0:
                monkeys[monkeys[monkey]['true']]['items'].append(new)
            else:
                monkeys[monkeys[monkey]['false']]['items'].append(new)

            monkeys[monkey]['inspected'] += 1

In [11]:
np.prod(sorted((m['inspected'] for m in monkeys.values()), reverse=True)[:2])

50616

In [25]:
monkeys = {}

for (monkey, items, operation, test, true, false) in pattern.findall(sample):
    monkeys[int(monkey)] = {'items': [], 'operation': parse_operation(operation), 
                            'test': int(test), 'true': int(true), 
                            'false': int(false), 'inspected': 0}
    for item in items.split(', '):
        monkeys[int(monkey)]['items'].append(int(item))

In [26]:
monkeys

{0: {'items': [79, 98],
  'operation': <function __main__.parse_operation.<locals>.<lambda>(x)>,
  'test': 23,
  'true': 2,
  'false': 3,
  'inspected': 0},
 1: {'items': [54, 65, 75, 74],
  'operation': <function __main__.parse_operation.<locals>.<lambda>(x)>,
  'test': 19,
  'true': 2,
  'false': 0,
  'inspected': 0},
 2: {'items': [79, 60, 97],
  'operation': <function __main__.parse_operation.<locals>.<lambda>(x)>,
  'test': 13,
  'true': 1,
  'false': 3,
  'inspected': 0},
 3: {'items': [74],
  'operation': <function __main__.parse_operation.<locals>.<lambda>(x)>,
  'test': 17,
  'true': 0,
  'false': 1,
  'inspected': 0}}

In [27]:
for _ in tqdm(range(10000)):
    for monkey, monkey_attrs in monkeys.items():
        for old in monkey_attrs['items']:
            new = monkey_attrs['operation'](old)
            if new % monkey_attrs['test'] == 0:
                monkeys[monkey_attrs['true']]['items'].append(new)
            else:
                monkeys[monkey_attrs['false']]['items'].append(new)

            monkey_attrs['inspected'] += 1

        monkey_attrs['items'].clear()

  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
np.prod(sorted((m['inspected'] for m in monkeys.values()), reverse=True)[:2])

In [ ]:
print('\n'.join([m['operation'] for m in monkeys.values()]))